# 1. Set Up Your Environment

In [1]:
# Authenticate with GCP
# !gcloud auth application-default login
!gcloud config set project eikon-dev-ai-team

Updated property [core/project].


In [2]:
!gcloud config get-value project

eikon-dev-ai-team


In [3]:
# Setup GCP project and location

PROJECT_ID = "eikon-dev-ai-team"  
LOCATION = "us-central1"  
DATASET_ID = 'test1' 

In [4]:
# Import vertexai Library
import base64
import vertexai
from vertexai.generative_models import GenerativeModel, SafetySetting, Part

vertexai.init(project=PROJECT_ID, location=LOCATION)

In [5]:
# Import bigquery library
from google.cloud import bigquery
import pandas as pd

bq_client = bigquery.Client(project=PROJECT_ID)
client = bq_client

# Testing the API

In [6]:
from setup import multiturn_generate_content, get_available_dataset, get_columns_from_datasets, generate_sql_query, clean_generated_query, execute_generated_query

In [7]:
# Get available dataset
get_available_dataset(PROJECT_ID)

h:\My Drive\gemini-to-sql\.venv\lib\site-packages\google\cloud\bigquery\table.py:1727: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


Available dataset saved to json-files\dataset.json


,schema_name
0,medicare_test
1,test1
2,country_hits10
3,logistic_company


In [8]:
json_file = "json-files/dataset.json"
get_columns_from_datasets(json_file, PROJECT_ID, output_folder="json-files", output_file="columns.json")

Columns saved to json-files\columns.json


{'medicare_test': {'inpatient_charges_2013': {'drg_definition': 'STRING',
   'provider_id': 'INT64',
   'provider_name': 'STRING',
   'provider_street_address': 'STRING',
   'provider_city': 'STRING',
   'provider_state': 'STRING',
   'provider_zipcode': 'INT64',
   'hospital_referral_region_description': 'STRING',
   'total_discharges': 'INT64',
   'average_covered_charges': 'FLOAT64',
   'average_total_payments': 'FLOAT64',
   'average_medicare_payments': 'FLOAT64',
   'year': 'INT64'},
  'inpatient_charges_2014': {'drg_definition': 'STRING',
   'provider_id': 'INT64',
   'provider_name': 'STRING',
   'provider_street_address': 'STRING',
   'provider_city': 'STRING',
   'provider_state': 'STRING',
   'provider_zipcode': 'INT64',
   'hospital_referral_region_description': 'STRING',
   'total_discharges': 'INT64',
   'average_covered_charges': 'FLOAT64',
   'average_total_payments': 'FLOAT64',
   'average_medicare_payments': 'FLOAT64',
   'year': 'INT64'},
  'outpatient_charges_2012': 

In [10]:
folder_path = "json-files"
user_request = "Create a query focuses on selecting specific information (columns) about medical providers in Alaska (AK) and ranks them based on the highest average total payments for inpatient services. Displays only the top 10 providers by this ranking. Only for year 2012"

generated_query = generate_sql_query(folder_path, user_request)
cleaned_query = clean_generated_query(generated_query)

In [12]:
print(generated_query)

["```sql\nSELECT\n    provider_id,\n    provider_name,\n    provider_state,\n    average_total_payments\n  FROM\n    `medicare_test.inpatient_charges_2012`\n  WHERE provider_state = 'AK'\nORDER BY\n  average_total_payments DESC\nLIMIT 10\n```\n"]


In [11]:
execute_generated_query(PROJECT_ID,cleaned_query)

h:\My Drive\gemini-to-sql\.venv\lib\site-packages\google\cloud\bigquery\table.py:1727: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


Query executed successfully. 10 rows retrieved.


,provider_id,provider_name,provider_state,average_total_payments
0,20001,PROVIDENCE ALASKA MEDICAL CENTER,AK,60365.21429
1,20001,PROVIDENCE ALASKA MEDICAL CENTER,AK,58481.12500
2,20001,PROVIDENCE ALASKA MEDICAL CENTER,AK,34464.07143
3,20001,PROVIDENCE ALASKA MEDICAL CENTER,AK,29538.57143
4,20001,PROVIDENCE ALASKA MEDICAL CENTER,AK,26220.79310
5,20001,PROVIDENCE ALASKA MEDICAL CENTER,AK,23851.07895
6,20001,PROVIDENCE ALASKA MEDICAL CENTER,AK,22579.83333
7,20001,PROVIDENCE ALASKA MEDICAL CENTER,AK,18929.75743
8,20001,PROVIDENCE ALASKA MEDICAL CENTER,AK,18846.62500
9,20001,PROVIDENCE ALASKA MEDICAL CENTER,AK,18506.53571
